# Fetch Rewards Coding Exercise - Analytics Engineer

## First: Review unstructured JSON data and diagram a new structured relational data model

The aim is to convert the unstructured JSON files into cleaned structured data which we can query. Using pandas to process and clean the data.

In [54]:
import gzip
import shutil
import os
import pandas as pd
from ast import literal_eval
import json
from datetime import datetime
from sqlalchemy import create_engine

Unzipping the jsons

In [55]:
# Decompress gzip files if any
for i in os.listdir():
    if i.endswith('.json.gz'):
        with gzip.open(i, 'rb') as f_in:
            with open(i.replace('.gz', ''), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

In [56]:
# Function to clean the JSON file
def clean_json_file(input_file, output_file):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for line in f_in:
            line = line.strip()
            if line.startswith('{') and line.endswith('}'):
                f_out.write(line + '\n')

clean_json_file('users.json', 'cleaned_users.json')


receipts = pd.read_json('receipts.json', lines=True)
brands = pd.read_json('brands.json', lines=True)
users = pd.read_json('cleaned_users.json', lines=True)

print(receipts.head())
print(brands.head())
print(users.head())


                                    _id  bonusPointsEarned  \
0  {'$oid': '5ff1e1eb0a720f0523000575'}              500.0   
1  {'$oid': '5ff1e1bb0a720f052300056b'}              150.0   
2  {'$oid': '5ff1e1f10a720f052300057a'}                5.0   
3  {'$oid': '5ff1e1ee0a7214ada100056f'}                5.0   
4  {'$oid': '5ff1e1d20a7214ada1000561'}                5.0   

                             bonusPointsEarnedReason  \
0  Receipt number 2 completed, bonus point schedu...   
1  Receipt number 5 completed, bonus point schedu...   
2                         All-receipts receipt bonus   
3                         All-receipts receipt bonus   
4                         All-receipts receipt bonus   

                 createDate               dateScanned  \
0  {'$date': 1609687531000}  {'$date': 1609687531000}   
1  {'$date': 1609687483000}  {'$date': 1609687483000}   
2  {'$date': 1609687537000}  {'$date': 1609687537000}   
3  {'$date': 1609687534000}  {'$date': 1609687534000}   
4  {'

### Receipts table

In [57]:
receipts.head()

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptItemList,rewardsReceiptStatus,totalSpent,userId
0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,{'$date': 1609632000000},5.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,26.0,5ff1e1eacfcf6c399c274ae6
1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,{'$date': 1609601083000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,11.0,5ff1e194b6a9d73a3a9f1052
2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,{'$date': 1609632000000},1.0,"[{'needsFetchReview': False, 'partnerItemId': ...",REJECTED,10.0,5ff1e1f1cfcf6c399c274b0b
3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,{'$date': 1609632000000},4.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,28.0,5ff1e1eacfcf6c399c274ae6
4,{'$oid': '5ff1e1d20a7214ada1000561'},5.0,All-receipts receipt bonus,{'$date': 1609687506000},{'$date': 1609687506000},{'$date': 1609687511000},{'$date': 1609687511000},{'$date': 1609687506000},5.0,{'$date': 1609601106000},2.0,"[{'barcode': '4011', 'description': 'ITEM NOT ...",FINISHED,1.0,5ff1e194b6a9d73a3a9f1052


The next step is to explode the 'rewardsReceiptItemList' field as it contains lists of receipts(we do this using literal_eval function from the ast package). Then we need to split each dictionary and get the columns from it(we do this using the json_normalize function in pandas). Once we do this we join this data back to the receipts data frame.

In [58]:
receipts = receipts.explode('rewardsReceiptItemList')
receipts.reset_index(inplace=True)

In [59]:
receipts = receipts.fillna({'rewardsReceiptItemList':'{}'})
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(lambda x:str(x))
receipts['rewardsReceiptItemList'] = receipts['rewardsReceiptItemList'].apply(literal_eval)

In [60]:
receipts_norm = pd.json_normalize(receipts['rewardsReceiptItemList'],errors='ignore',record_prefix='rewardsReceiptItemList')\
.add_prefix('rewardsReceiptItemList.')

In [61]:
receipts_final = pd.merge(receipts, receipts_norm, left_index=True, right_index=True, how='outer')

In [62]:
receipts_final

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,{'$oid': '5ff1e1eb0a720f0523000575'},500.0,"Receipt number 2 completed, bonus point schedu...",{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687531000},{'$date': 1609687536000},{'$date': 1609687531000},500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,{'$oid': '5ff1e1bb0a720f052300056b'},150.0,"Receipt number 5 completed, bonus point schedu...",{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687483000},{'$date': 1609687488000},{'$date': 1609687483000},150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,{'$oid': '5ff1e1f10a720f052300057a'},5.0,All-receipts receipt bonus,{'$date': 1609687537000},{'$date': 1609687537000},NaN,{'$date': 1609687542000},NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,{'$oid': '5ff1e1ee0a7214ada100056f'},5.0,All-receipts receipt bonus,{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687534000},{'$date': 1609687539000},{'$date': 1609687534000},5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7376,1115,{'$oid': '603d0b710a720fde1000042a'},NaN,NaN,{'$date': 1614613361873},{'$date': 1614613361873},NaN,{'$date': 1614613361873},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7377,1116,{'$oid': '603cf5290a720fde10000413'},NaN,NaN,{'$date': 1614607657664},{'$date': 1614607657664},NaN,{'$date': 1614607657664},NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7378,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
7379,1117,{'$oid': '603ce7100a7217c72c000405'},25.0,COMPLETE_NONPARTNER_RECEIPT,{'$date': 1614604048000},{'$date': 1614604048000},NaN,{'$date': 1614604049000},NaN,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN


Now that we have exploded 'rewardsReceiptItemList' into multiple rows and split the keys into different columns we need to format the other fields that have dictionaries.

In [63]:
def extract_keys_from_columns(df, columns):
    keys_dict = {}
    for column in columns:
        if column in df.columns:
            keys_set = set()
            # Drop NaN values for the current column
            column_data = df[column].dropna()
            for item in column_data:
                if isinstance(item, dict):
                    keys_set.update(item.keys())
            keys_dict[column] = keys_set
    return keys_dict


columns_to_process = ['_id', 'createDate', 'dateScanned', 'finishedDate', 'modifyDate', 'pointsAwardedDate', 'purchaseDate']


keys_dict = extract_keys_from_columns(receipts_final, columns_to_process)


for column, keys in keys_dict.items():
    print(f"Keys in column '{column}': {keys}")


Keys in column '_id': {'$oid'}
Keys in column 'createDate': {'$date'}
Keys in column 'dateScanned': {'$date'}
Keys in column 'finishedDate': {'$date'}
Keys in column 'modifyDate': {'$date'}
Keys in column 'pointsAwardedDate': {'$date'}
Keys in column 'purchaseDate': {'$date'}


We see that each of these columns contain only one dictionary item each so we just need to extract these fields. As for the date columns the time is stored as UTC so we convert that into datetime format using the apply function.

In [64]:
def date_converter(x):
  try:
    return(datetime.utcfromtimestamp(int(x['$date'])/1000).strftime('%Y-%m-%d %H:%M:%S'))
  except TypeError:
    return(None)


In [65]:
receipts_final['_id'] = receipts_final['_id'].apply(lambda x: x['$oid'])
receipts_final['createDate'] = receipts_final['createDate'].apply(lambda x: date_converter(x))
receipts_final['dateScanned'] = receipts_final['dateScanned'].apply(lambda x: date_converter(x))
receipts_final['finishedDate'] = receipts_final['finishedDate'].apply(lambda x: date_converter(x))
receipts_final['modifyDate'] = receipts_final['modifyDate'].apply(lambda x: date_converter(x))
receipts_final['pointsAwardedDate'] = receipts_final['pointsAwardedDate'].apply(lambda x: date_converter(x))
receipts_final['purchaseDate'] = receipts_final['purchaseDate'].apply(lambda x: date_converter(x))

In [66]:
receipts_final.head()

,index,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,...,rewardsReceiptItemList.itemNumber,rewardsReceiptItemList.originalMetaBriteQuantityPurchased,rewardsReceiptItemList.pointsEarned,rewardsReceiptItemList.targetPrice,rewardsReceiptItemList.competitiveProduct,rewardsReceiptItemList.originalFinalPrice,rewardsReceiptItemList.originalMetaBriteItemPrice,rewardsReceiptItemList.deleted,rewardsReceiptItemList.priceAfterCoupon,rewardsReceiptItemList.metabriteCampaignId
0,0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,None,2021-01-03 15:25:42,None,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Users table

In [67]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
3,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [68]:
def extract_keys_from_column(df_column):
    keys_set = set()
    for item in df_column.dropna():
        if isinstance(item, dict):
            keys_set.update(item.keys())
    return keys_set

# Extract keys from specified columns
id_keys = extract_keys_from_column(users['_id'])
createdDate_keys = extract_keys_from_column(users['createdDate'])
lastLogin_keys = extract_keys_from_column(users['lastLogin'])


print(f"Keys in '_id' column: {id_keys}")
print(f"Keys in 'createdDate' column: {createdDate_keys}")
print(f"Keys in 'lastLogin' column: {lastLogin_keys}")


Keys in '_id' column: {'$oid'}
Keys in 'createdDate' column: {'$date'}
Keys in 'lastLogin' column: {'$date'}


Here we see that each dictionary contains just one key each and we just need to extract the value and we will use the same function created earlier to convert date into a user friendly format.

In [69]:
users['_id'] = users['_id'].apply(lambda x: x['$oid'])
users['createdDate'] = users['createdDate'].apply(lambda x: date_converter(x))
users['lastLogin'] = users['lastLogin'].apply(lambda x: date_converter(x))

In [70]:
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
2,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,Email,WI
3,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,Email,WI


## Brands table

In [71]:
brands.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode
0,{'$oid': '601ac115be37ce2ead437551'},511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN
1,{'$oid': '601c5460be37ce2ead43755f'},511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS
2,{'$oid': '601ac142be37ce2ead43755d'},511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176
3,{'$oid': '601ac142be37ce2ead43755a'},511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051
4,{'$oid': '601ac142be37ce2ead43755e'},511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827


In [72]:

set().union(*(d.keys() for d in brands['cpg']))

{'$id', '$ref'}

We need to explode the cpg fields as there is more than one element in the dictionary.

In [73]:
brands_norm = pd.json_normalize(brands['cpg'])
brands_norm = brands_norm.add_prefix('cpg.')

In [74]:
brands_norm

,cpg.$ref,cpg.$id.$oid
0,Cogs,601ac114be37ce2ead437550
1,Cogs,5332f5fbe4b03c9a25efd0ba
2,Cogs,601ac142be37ce2ead437559
3,Cogs,601ac142be37ce2ead437559
4,Cogs,5332fa12e4b03c9a25efd1e7
...,...,...
1162,Cogs,5f77274dbe37ce6b592e90bf
1163,Cogs,53e10d6368abd3c7065097cc
1164,Cogs,5332fa12e4b03c9a25efd1e7
1165,Cogs,5332f5f6e4b03c9a25efd0b4


In [75]:
brands_final = pd.merge(brands, brands_norm, left_index=True, right_index=True, how='outer')

In [76]:
brands_final.shape

(1167, 10)

In [77]:
brands_final['_id'] = brands_final['_id'].apply(lambda x: x['$oid'])

In [78]:
brands_final.head()

,_id,barcode,category,categoryCode,cpg,name,topBrand,brandCode,cpg.$ref,cpg.$id.$oid
0,601ac115be37ce2ead437551,511111019862,Baking,BAKING,"{'$id': {'$oid': '601ac114be37ce2ead437550'}, ...",test brand @1612366101024,0.0,NaN,Cogs,601ac114be37ce2ead437550
1,601c5460be37ce2ead43755f,511111519928,Beverages,BEVERAGES,"{'$id': {'$oid': '5332f5fbe4b03c9a25efd0ba'}, ...",Starbucks,0.0,STARBUCKS,Cogs,5332f5fbe4b03c9a25efd0ba
2,601ac142be37ce2ead43755d,511111819905,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146176,0.0,TEST BRANDCODE @1612366146176,Cogs,601ac142be37ce2ead437559
3,601ac142be37ce2ead43755a,511111519874,Baking,BAKING,"{'$id': {'$oid': '601ac142be37ce2ead437559'}, ...",test brand @1612366146051,0.0,TEST BRANDCODE @1612366146051,Cogs,601ac142be37ce2ead437559
4,601ac142be37ce2ead43755e,511111319917,Candy & Sweets,CANDY_AND_SWEETS,"{'$id': {'$oid': '5332fa12e4b03c9a25efd1e7'}, ...",test brand @1612366146827,0.0,TEST BRANDCODE @1612366146827,Cogs,5332fa12e4b03c9a25efd1e7


## Second: Write a query that directly answers a predetermined question from a business stakeholder

I am using SQLite database through [SQL Alchemy](https://docs.sqlalchemy.org/en/14/dialects/sqlite.html) python to implement the SQL database for the queries.

In [79]:
from sqlalchemy import create_engine
import pandas as pd


# Create SQLite engine
engine = create_engine('sqlite://', echo=False)

# Clean and save 'users' DataFrame to SQLite
users.drop_duplicates(subset=['_id'], inplace=True)
users.to_sql('users', con=engine, index=False, if_exists='replace')

# Clean and save 'receipts_final' DataFrame to SQLite
receipts_final.columns = receipts_final.columns.str.replace('.', '_')
receipts_final.drop(columns=['rewardsReceiptItemList'], inplace=True)
receipts_final.to_sql('receipts', con=engine, index=False, if_exists='replace')


brands_final.drop_duplicates(subset=['brandCode'], inplace=True)
brands_final.drop(columns=['cpg'], inplace=True)
brands_final.to_sql('brands', con=engine, index=False, if_exists='replace')


with engine.connect() as connection:
    users_data = pd.read_sql('users', connection)
    receipts_data = pd.read_sql('receipts', connection)
    brands_data = pd.read_sql('brands', connection)

print("Users Data:")
print(users_data.head())
print("\nReceipts Data:")
print(receipts_data.head())
print("\nBrands Data:")
print(brands_data.head())


Users Data:
                        _id  active          createdDate            lastLogin  \
0  5ff1e194b6a9d73a3a9f1052    True  2021-01-03 15:24:04  2021-01-03 15:25:37   
1  5ff1e1eacfcf6c399c274ae6    True  2021-01-03 15:25:30  2021-01-03 15:25:30   
2  5ff1e1e8cfcf6c399c274ad9    True  2021-01-03 15:25:28  2021-01-03 15:25:28   
3  5ff1e1b7cfcf6c399c274a5a    True  2021-01-03 15:24:39  2021-01-03 15:24:39   
4  5ff1e1f1cfcf6c399c274b0b    True  2021-01-03 15:25:37  2021-01-03 15:25:37   

       role signUpSource state  
0  consumer        Email    WI  
1  consumer        Email    WI  
2  consumer        Email    WI  
3  consumer        Email    WI  
4  consumer        Email    WI  

Receipts Data:
   index                       _id  bonusPointsEarned  \
0      0  5ff1e1eb0a720f0523000575              500.0   
1      1  5ff1e1bb0a720f052300056b              150.0   
2      1  5ff1e1bb0a720f052300056b              150.0   
3      2  5ff1e1f10a720f052300057a                5.0   
4 

In [80]:
print("Users Data Columns:")
print(users_data.columns)
print("\nReceipts Data Columns:")
print(receipts_data.columns)
print("\nBrands Data Columns:")
print(brands_data.columns)

Users Data Columns:
Index(['_id', 'active', 'createdDate', 'lastLogin', 'role', 'signUpSource',
       'state'],
      dtype='object')

Receipts Data Columns:
Index(['index', '_id', 'bonusPointsEarned', 'bonusPointsEarnedReason',
       'createDate', 'dateScanned', 'finishedDate', 'modifyDate',
       'pointsAwardedDate', 'pointsEarned', 'purchaseDate',
       'purchasedItemCount', 'rewardsReceiptStatus', 'totalSpent', 'userId',
       'rewardsReceiptItemList_barcode', 'rewardsReceiptItemList_description',
       'rewardsReceiptItemList_finalPrice', 'rewardsReceiptItemList_itemPrice',
       'rewardsReceiptItemList_needsFetchReview',
       'rewardsReceiptItemList_partnerItemId',
       'rewardsReceiptItemList_preventTargetGapPoints',
       'rewardsReceiptItemList_quantityPurchased',
       'rewardsReceiptItemList_userFlaggedBarcode',
       'rewardsReceiptItemList_userFlaggedNewItem',
       'rewardsReceiptItemList_userFlaggedPrice',
       'rewardsReceiptItemList_userFlaggedQuantity

**_Q1) What are the top 5 brands by receipts scanned for most recent month?_**


In [81]:
sql_query = """
WITH brand_receipts AS (
    SELECT
        COALESCE(b.name, 'N/A') AS brand_name,
        STRFTIME('%Y-%m', r.purchaseDate) AS receipt_month,
        COUNT(DISTINCT r._id) AS count_of_receipts
    FROM receipts r
    LEFT JOIN brands b ON r.rewardsReceiptItemList_brandCode = b.brandCode
    WHERE r.purchaseDate >= STRFTIME('%Y-%m-01', DATETIME('now', '-48 months'))
    GROUP BY 1, 2
),
total_receipts AS (
    SELECT
        brand_name,
        SUM(count_of_receipts) AS total_count_of_receipts
    FROM brand_receipts
    GROUP BY 1
),
ranked_brands AS (
    SELECT
        brand_name,
        total_count_of_receipts,
        ROW_NUMBER() OVER (ORDER BY total_count_of_receipts DESC) AS rank
    FROM total_receipts
    WHERE brand_name != 'N/A'
)
SELECT
    brand_name,
    total_count_of_receipts,
    rank
FROM ranked_brands
WHERE rank <= 5
ORDER BY rank;
"""


In [82]:
result_df = pd.read_sql_query(sql_query, engine)


In [83]:
print(result_df)


  brand_name  total_count_of_receipts  rank
0      Pepsi                       23     1
1      Kraft                       22     2
2    Kleenex                       21     3
3    Doritos                       19     4
4      KNORR                       19     5


In [ ]:
When I initially executed the query the highest total_count_of_receipts was for 'None' brand_name which does not make 
any sense. Hence, I have excluded none and counted the rest of them.
This seems to be an error that must have occured during data collection or during data entry while creating jsons.

**_Q2) How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month?_**

In [84]:
# Query to get the minimum and maximum purchaseDate in the receipts table
date_range_query = """
SELECT
    MIN(purchaseDate) AS min_date,
    MAX(purchaseDate) AS max_date
FROM receipts;
"""

with engine.connect() as connection:
    date_range_df = pd.read_sql_query(date_range_query, connection)

print("Date range in the receipts table:")
print(date_range_df)


Date range in the receipts table:
              min_date             max_date
0  2017-10-30 00:00:00  2021-03-08 17:37:13


In [33]:
# Query to sample some dates to check their format
sample_dates_query = """
SELECT
    purchaseDate
FROM receipts
LIMIT 10;
"""

with engine.connect() as connection:
    sample_dates_df = pd.read_sql_query(sample_dates_query, connection)

print("Sample dates from the receipts table:")
print(sample_dates_df)


Sample dates from the receipts table:
          purchaseDate
0  2021-01-03 00:00:00
1  2021-01-02 15:24:43
2  2021-01-02 15:24:43
3  2021-01-03 00:00:00
4  2021-01-03 00:00:00
5  2021-01-02 15:25:06
6  2021-01-02 15:25:06
7  2021-01-02 15:25:24
8  2021-01-03 15:25:01
9  2020-12-27 00:00:00


In [34]:
# Query to check if there are any records in the past few months
past_few_months_query = """
SELECT
    STRFTIME('%Y-%m', purchaseDate) AS receipt_month,
    COUNT(*)
FROM receipts
GROUP BY receipt_month
ORDER BY receipt_month DESC
LIMIT 6;
"""

with engine.connect() as connection:
    past_few_months_df = pd.read_sql_query(past_few_months_query, connection)

print("Receipts count for the past few months:")
print(past_few_months_df)


Receipts count for the past few months:
  receipt_month  COUNT(*)
0       2021-03         2
1       2021-02       172
2       2021-01      6492
3       2020-12        86
4       2020-11        54
5       2020-10        16


In [35]:
# Join the recent month and previous month dataframes
comparison_query = """
WITH recent_month AS (
    SELECT
        COALESCE(b.name, 'N/A') AS brand_name,
        COUNT(DISTINCT r._id) AS count_of_receipts,
        ROW_NUMBER() OVER (ORDER BY COUNT(DISTINCT r._id) DESC) AS rank
    FROM receipts r
    LEFT JOIN brands b ON r.rewardsReceiptItemList_brandCode = b.brandCode
    WHERE STRFTIME('%Y-%m', r.purchaseDate) = '2021-01'
    GROUP BY brand_name
    HAVING brand_name != 'N/A'
),
previous_month AS (
    SELECT
        COALESCE(b.name, 'N/A') AS brand_name,
        COUNT(DISTINCT r._id) AS count_of_receipts,
        ROW_NUMBER() OVER (ORDER BY COUNT(DISTINCT r._id) DESC) AS rank
    FROM receipts r
    LEFT JOIN brands b ON r.rewardsReceiptItemList_brandCode = b.brandCode
    WHERE STRFTIME('%Y-%m', r.purchaseDate) = '2020-12'
    GROUP BY brand_name
    HAVING brand_name != 'N/A'
)
SELECT
    recent.brand_name AS recent_brand_name,
    recent.count_of_receipts AS recent_count_of_receipts,
    recent.rank AS recent_rank,
    previous.brand_name AS previous_brand_name,
    previous.count_of_receipts AS previous_count_of_receipts,
    previous.rank AS previous_rank
FROM recent_month recent
LEFT JOIN previous_month previous
ON recent.brand_name = previous.brand_name
ORDER BY recent_rank;
"""

comparison_df = pd.read_sql_query(comparison_query, engine)
print("Comparison of top 5 brands by receipts scanned for January 2021 vs December 2020:")
print(comparison_df)


Comparison of top 5 brands by receipts scanned for January 2021 vs December 2020:
                 recent_brand_name  recent_count_of_receipts  recent_rank  \
0                            Pepsi                        23            1   
1                            Kraft                        22            2   
2                          Kleenex                        21            3   
3                            KNORR                        19            4   
4                          Doritos                        19            5   
5                      Rice A Roni                        14            6   
6    Mayo by HELLMANN'S/BEST FOODS                        12            7   
7            Cracker Barrel Cheese                        12            8   
8                     Yuban Coffee                        11            9   
9                         Tostitos                        11           10   
10                         Swanson                        11           

**_Q3) When considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?_**

In [37]:
# Execute the query to compare average spend for 'Accepted' and 'Rejected' receipts
query = """
WITH Receipts_Agg AS (
    SELECT rewardsReceiptStatus, AVG(totalSpent) AS avg_total_spent
    FROM receipts
    WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
    GROUP BY rewardsReceiptStatus
)
SELECT
    rewardsReceiptStatus,
    AVG(avg_total_spent) AS average_spend
FROM Receipts_Agg
GROUP BY rewardsReceiptStatus;
"""

result_df = pd.read_sql_query(query, engine)

print("Comparison of average spend from receipts with 'rewardsReceiptStatus' of 'Accepted' or 'Rejected':")
for index, row in result_df.iterrows():
    print(f"{row['rewardsReceiptStatus']}: ${row['average_spend']:.2f}")

# Determine which average spend is greater
if len(result_df) == 2:
    if result_df.iloc[0]['average_spend'] > result_df.iloc[1]['average_spend']:
        print("Average spend from 'Accepted' receipts is greater.")
    elif result_df.iloc[0]['average_spend'] < result_df.iloc[1]['average_spend']:
        print("Average spend from 'Rejected' receipts is greater.")
    else:
        print("Average spend from 'Accepted' and 'Rejected' receipts is the same.")
else:
    print("Only one status ('Accepted' or 'Rejected') present in the data.")

Comparison of average spend from receipts with 'rewardsReceiptStatus' of 'Accepted' or 'Rejected':
Only one status ('Accepted' or 'Rejected') present in the data.


In [38]:
query = """
SELECT
    rewardsReceiptStatus,
    AVG(totalSpent) AS average_spend
FROM receipts
GROUP BY rewardsReceiptStatus;
"""

result_df = pd.read_sql_query(query, engine)


print("Comparison of average spend from receipts with all 'rewardsReceiptStatus':")
for index, row in result_df.iterrows():
    print(f"{row['rewardsReceiptStatus']}: ${row['average_spend']:.2f}")

Comparison of average spend from receipts with all 'rewardsReceiptStatus':
FINISHED: $1244.37
FLAGGED: $2635.57
PENDING: $28.03
REJECTED: $19.54
SUBMITTED: $nan


In [ ]:
Receipts with status of FINISHED/ACCEPTED have a higher spend than the rest.

Upon reviewing the data, we haven't found any receipts with the status "ACCEPTED". 
The closest status we have is "FINISHED". While "FINISHED" may have a similar meaning to "ACCEPTED", 
it's possible that it represents a different state entirely. This inconsistency needs attention and should be 
addressed promptly.

Action Needed:

1.Clarification Needed: Determine the exact meaning of "FINISHED" status in the receipts data.
    It may or may not represent the same state as "ACCEPTED".

2.Data Consistency Check: Verify if there are other status values that are equivalent to "ACCEPTED" or if 
    "FINISHED" indeed serves the same purpose.

3.Update Documentation: If "FINISHED" is confirmed to be equivalent to "ACCEPTED", update the documentation to
    reflect this understanding. Otherwise, investigate further to determine the correct status.

4.Data Cleaning: If necessary, clean the data to ensure consistency in status values.

5.Long-term Solution: Implement checks to ensure data consistency from the source to avoid similar issues in the 
    future.

**_Q4) When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?_**

In [ ]:
query = """
WITH Receipts_Agg AS (
    SELECT rewardsReceiptStatus, SUM(purchasedItemCount) AS total_items_purchased
    FROM receipts
    WHERE rewardsReceiptStatus IN ('Accepted', 'Rejected')
    GROUP BY rewardsReceiptStatus
)
SELECT
    rewardsReceiptStatus,
    SUM(total_items_purchased) AS total_items_purchased
FROM Receipts_Agg
GROUP BY rewardsReceiptStatus;
"""

result_df = pd.read_sql_query(query, engine)


print("Comparison of total number of items purchased from receipts with 'rewardsReceiptStatus' of 'Accepted' or 'Rejected':")
for index, row in result_df.iterrows():
    print(f"{row['rewardsReceiptStatus']}: {int(row['total_items_purchased'])} items")

# Determine which total is greater
if len(result_df) == 2:
    if result_df.iloc[0]['total_items_purchased'] > result_df.iloc[1]['total_items_purchased']:
        print("Total number of items purchased from 'Accepted' receipts is greater.")
    elif result_df.iloc[0]['total_items_purchased'] < result_df.iloc[1]['total_items_purchased']:
        print("Total number of items purchased from 'Rejected' receipts is greater.")
    else:
        print("Total number of items purchased from 'Accepted' and 'Rejected' receipts is the same.")
else:
    print("Only one status ('Accepted' or 'Rejected') present in the data.")

In [41]:
query = """
SELECT
    rewardsReceiptStatus,
    SUM(purchasedItemCount) AS total_items_purchased
FROM receipts
GROUP BY rewardsReceiptStatus;
"""

result_df = pd.read_sql_query(query, engine)

# Fill NaN values with 0
result_df['total_items_purchased'] = result_df['total_items_purchased'].fillna(0)


print("Comparison of total number of items purchased from receipts with all 'rewardsReceiptStatus':")
for index, row in result_df.iterrows():
    print(f"{row['rewardsReceiptStatus']}: {int(row['total_items_purchased'])} items")

Comparison of total number of items purchased from receipts with all 'rewardsReceiptStatus':
FINISHED: 1364998 items
FLAGGED: 294816 items
PENDING: 0 items
REJECTED: 740 items
SUBMITTED: 0 items


**_Q5) Which brand has the most spend among users who were created within the past 6 months?_**

In [48]:
query = """
SELECT COUNT(*) AS total_users, MIN(createdDate) AS min_created_date, MAX(createdDate) AS max_created_date
FROM Users;
"""

# Execute the query and load result into a DataFrame
result_df = pd.read_sql_query(query, engine)


print("Summary of Users table:")
print(result_df)

Summary of Users table:
   total_users     min_created_date     max_created_date
0          212  2014-12-19 14:21:22  2021-02-12 14:11:06


In [51]:
# Define the query to select users created in the previous 6 months from '2021-02-12 14:11:06' and their spending per brand
query = """
WITH RecentUsers AS (
    SELECT _id
    FROM Users
    WHERE createdDate >= DATE('2021-02-12 14:11:06', '-6 months')
    AND createdDate < '2021-02-12 14:11:06'
),
UserReceipts AS (
    SELECT r._id, r.totalSpent, r.userId
    FROM Receipts r
    JOIN RecentUsers u ON r.userId = u._id
),
BrandSpending AS (
    SELECT b.name AS brandName, SUM(r.totalSpent) AS totalSpent
    FROM UserReceipts r
    JOIN Receipts rr ON r._id = rr._id
    LEFT JOIN Brands b ON rr.rewardsReceiptItemList_barcode = b.barcode
    WHERE b.name IS NOT NULL
    GROUP BY b.name
)
SELECT brandName, MAX(totalSpent) AS totalSpent
FROM BrandSpending
GROUP BY brandName
ORDER BY totalSpent DESC
LIMIT 1;
"""

# Execute the query and load result into a DataFrame
result_df = pd.read_sql_query(query, engine)


print("Brand with the most spend among users created in the past 6 months:")
print(result_df)

Brand with the most spend among users created in the previous 6 months:
         brandName  totalSpent
0  Pepperidge Farm   6374632.5


**_Q6) Which brand has the most transactions among users who were created within the past 6 months?_**

In [53]:
# Define the query to count transactions per brand for users created in the past 6 months
query = """
WITH RecentUsers AS (
    SELECT _id
    FROM Users
    WHERE createdDate >= DATE('2021-02-12 14:11:06', '-6 months')
    AND createdDate < '2021-02-12 14:11:06'
),
UserReceipts AS (
    SELECT r._id, r.userId
    FROM Receipts r
    JOIN RecentUsers u ON r.userId = u._id
),
BrandTransactions AS (
    SELECT b.name AS brandName, COUNT(*) AS transactionCount
    FROM UserReceipts r
    JOIN Receipts rr ON r._id = rr._id
    LEFT JOIN Brands b ON rr.rewardsReceiptItemList_barcode = b.barcode
    WHERE b.name IS NOT NULL
    GROUP BY b.name
)
SELECT brandName, MAX(transactionCount) AS maxTransactionCount
FROM BrandTransactions
GROUP BY brandName
ORDER BY maxTransactionCount DESC
LIMIT 1;
"""

result_df = pd.read_sql_query(query, engine)

print("Brand with the most transactions among users created in the past 6 months:")
print(result_df)

Brand with the most transactions among users created in the past 6 months:
  brandName  maxTransactionCount
0  Tostitos                 3218


## Third: Evaluate Data Quality Issues in the Data Provided

1)To find data quality issues I decided to look into the mapping between the brands data and receipts data using brandCode.

In [88]:
len(set(receipts_final.dropna(subset=['rewardsReceiptItemList_brandCode'])['rewardsReceiptItemList_brandCode']))

227

In [89]:
len(set(brands_final.dropna(subset=['brandCode'])['brandCode']))

897

there are a lot more unique brand codes in the brands dataset compared to the receipts. Now we compare the two and see if all the receipts with a brand code can be mapped to the brands table.

In [ ]:
2)Missing Data

In [91]:
def percentage_missing_data(df):
    missing_data = df.isnull().sum()
    percentage_missing = (missing_data / df.shape[0]) * 100
    return percentage_missing
percentage_missing_data(users)

_id              0.000000
active           0.000000
createdDate      0.000000
lastLogin       18.867925
role             0.000000
signUpSource     2.358491
state            2.830189
dtype: float64

In [92]:
percentage_missing_data(brands)

_id              0.000000
barcode          0.000000
category        13.281919
categoryCode    55.698372
cpg              0.000000
name             0.000000
topBrand        52.442159
brandCode       20.051414
dtype: float64

In [93]:
percentage_missing_data(receipts_final)

index                                                         0.000000
_id                                                           0.000000
bonusPointsEarned                                            18.981168
bonusPointsEarnedReason                                      18.981168
createDate                                                    0.000000
dateScanned                                                   0.000000
finishedDate                                                 19.116651
modifyDate                                                    0.000000
pointsAwardedDate                                            17.626338
pointsEarned                                                 15.282482
purchaseDate                                                  6.205121
purchasedItemCount                                            6.557377
rewardsReceiptStatus                                          0.000000
totalSpent                                                    5.893510
userId

In [ ]:
3) Date format is not in timestamp in all the json files. Hence, used date converter function in above code 